In [1]:
include("../src/tools.jl")

get_next! (generic function with 1 method)

In [16]:
type Preferences
    size::Int
    prefs::AbstractArray
    caps::AbstractArray
end

type TwoSidedMatchingMarket
    side_A::Preferences
    side_B::Preferences
end


function make_ranking_from_prefs(proposer_size::Int, 
    non_proposer_size::Int, 
    non_proposer_prefs::AbstractArray)
    
    rankings = zeros(Int64, non_proposer_size, proposer_size)
    
    for non_prop in 1:non_proposer_size
        for (ranking, prop) in enumerate(non_proposer_prefs[non_prop])
            rankings[non_prop, prop] = ranking
        end
    end
    
    return rankings
end

"""
    deferred_acceptance(market[, reversed=false])

Carry out Deferred Acceptance algorithm(Gale-Shapley algorithm)

By default, side A offers. If `reversed==true`, then side B offers

For anyone with more capacity than one, I assume (implicitly) they have a
responsive preference.
"""
function deferred_acceptance(market::TwoSidedMatchingMarket, reversed=false)
    if reversed
        proposer_side = market.side_B
        non_proposer_side = market.side_A
    else
        proposer_side = market.side_A
        non_proposer_side = market.side_B
    end
    
    # ranking of proposers for each non-proposer
    rankings = make_ranking_from_prefs(proposer_side.size, non_proposer_side.size, non_proposer_side.prefs)
    
    # result matching container
    proposer_matched = Vector{Int64}(proposer_side.size)
    non_proposer_matched = zeros(Int64, non_proposer_side.size)
    
    # lowest ranking of non-proposer that each proposer has already proposed to
    proposed_counter = Counter([size(p, 1) for p in proposer_side.prefs])

    # stack of proposers who can still propose to someone
    proposer_stack = FixedSizeStack{Int64}(collect(proposer_side.size:-1:1), proposer_side.size)
    
    # DA loop
    while !isempty(proposer_stack)
        proposer = pop!(proposer_stack)
        
        while true
            next_ranking = get_next!(proposed_counter, proposer)
            if next_ranking == 0
                break
            end
                
            proposing_next = proposer_side.prefs[proposer][next_ranking]
            if rankings[proposing_next, proposer] > 0
                matched = non_proposer_matched[proposing_next]

                if matched == 0
                    proposer_matched[proposer] = proposing_next
                    non_proposer_matched[proposing_next] = proposer
                    break
                
                elseif rankings[proposing_next, proposer] < rankings[proposing_next, matched]
                    proposer_matched[proposer] = proposing_next
                    proposer_matched[matched] = 0
                    non_proposer_matched[proposing_next] = proposer
                    push!(proposer_stack, matched)
                    break
                end
            end
        end
    end
    
    return proposer_matched, non_proposer_matched
end

deferred_acceptance

In [15]:
m_size = 4
f_size = 3
m_prefs = [[3], [3, 2, 1], [1, 3, 2], [3, 1]]
f_prefs = [[2, 3], [2, 3, 4, 1], [4, 1, 2]]
m_caps = ones(Int64, m_size)
f_caps = ones(Int64, m_size)

m_side = Preferences(m_size, m_prefs, m_caps)
f_side = Preferences(f_size, f_prefs, f_caps)
market = TwoSidedMatchingMarket(m_side, f_side)

deferred_acceptance(market)

[0 1 2 0; 4 1 2 3; 2 3 0 1]
130
231
220
310
431


([0, 2, 1, 3], [3, 2, 4])